### Set Environment

In [2]:
import os
import json
import warnings
import pandas as pd
from glob2 import glob
warnings.filterwarnings("ignore")


### List all Output paths

In [6]:
jsonFiles = \
    glob('/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/*/posts/*.json'
         )  # Can be used absolute or relative paths


#### Create Aggregate folder

In [ ]:
# Ignore once done
os.makedirs("/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/aggregates_data")

### Function to
* __Merge all data in files__
* __Normalizes merged .json into dataframe__
* __Saves as csv__

In [8]:
tiktok_data = pd.DataFrame([])

def merge_JsonFiles(filename):
    for f1 in filename:
        global tiktok_data
        with open(f1, "r") as infile:
            data = json.load(infile)
        tiktok_data = tiktok_data.append(pd.json_normalize(data), ignore_index=True)
        tiktok_data.to_csv(
            "/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/aggregates_data/tiktok.csv",
            index=False,
        )

merge_JsonFiles(jsonFiles)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Cleaning
#### Delete duplicate data: by 'webVideoUrl'

In [9]:
# Delete duplicate rows based on specific columns

tiktok_data = tiktok_data.drop_duplicates(subset=['webVideoUrl'],
                                          keep=False)


### Data Summaries
#### To Flourish:

### Visuals
#### 1. User from 'webVideoUrl'

In [10]:
source = pd.DataFrame([])
source["source"] = tiktok_data['webVideoUrl'].str.split('/').str[3]
source["source"].value_counts().rename_axis("unique_sources").reset_index(
    name="counts"
).to_csv("/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/aggregates_data/source.csv")

#### 2. Virial Video: 'webvideoUrl' 

In [11]:
tiktok_data["videoUrl"].value_counts(ascending=True).rename_axis(
    "unique_videos"
).reset_index(name="counts").to_csv(
    "/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/aggregates_data/videos.csv"
)


#### 3. Network Analysis data
 * Source = 'authorMeta.name'
 * Target = 👇

In [12]:
pd.DataFrame(tiktok_data.mentions.values.tolist()).add_prefix('target_').to_csv(
    "/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/aggregates_data/target.csv"
)

#### Hashtags

In [33]:
#  key_hashtags:
with open('/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/tiktok_hashtag_analysis/hashtag_list.txt', "r") as f:
  key_hashtags = [ ]
  for item in f:
      key_hashtags.append(item.replace('\n', ''))

In [29]:
# Extracted hashtags:
pd.Series(
    [y.get("name") for x in tiktok_data["hashtags"] for y in x]
).value_counts().rename_axis("hashtag").reset_index(name="freq").to_csv(
    "/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/aggregates_data/hashtags.csv"
)

In [34]:
hashtags = pd.DataFrame([y.get("name") for x in tiktok_data["hashtags"] for y in x], columns=['Hash'])
expected_hashtags = hashtags[hashtags.Hash.isin(key_hashtags)]

In [36]:
# Save the major hashtags
expected_hashtags.value_counts().rename_axis("hashtag").reset_index(name="freq").to_csv(
    "/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/aggregates_data/required_hashtags.csv"
)

#### 3. Sentimental Analysis by 'text'